In [1]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score,log_loss
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
import math
%matplotlib inline

In [2]:
warnings.filterwarnings("ignore")   
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True,agrupamento=False)

In [3]:
k = 10
lista = list(range(0,len(matrix))) 
shuffle(lista)
a = np.asarray(lista) % k

In [4]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:3])
#matrixProcessed = preprocessing.normalize(matrix[:,:3], norm='l2')
#matrixProcessed = np.array([[matrixProcessed[y,0],matrixProcessed[y,1],matrixProcessed[y,2],matrixProcessed[y,3]]  for y in range(0,len(matrix[:,0]))])
matrixProcessed = matrixProcessed = np.array([[matrixProcessed[y,0],sigmoid(matrixProcessed[y,1]),matrixProcessed[y,2]]  for y in range(0,len(matrix[:,0]))])


In [20]:
def specificity(outputs,predicts):
    neg = np.sum([output == 0 for output in outputs])
    tn = np.sum([predicts[x] == outputs[x] and outputs[x] == 0 for x in range(len(predicts))])
    return tn / neg

def KFold(matrix,metric):
    k = 8
    
    limit = 3
    
    
    lista = list(range(1,len(matrix))) 
    shuffle(lista)
    a = np.asarray(lista) % k   
    
    scores = np.zeros(k)
    recall = np.zeros(k)
    precision = np.zeros(k)
    average_precision = np.zeros(k)
    
    for fold in range(0,k):
        treinoIndex = np.where(a != fold)[0]
        testeIndex =  np.where(a == fold)[0]        
        
        inputs = np.array([[matrix[y,0],matrix[y,1]] for y in treinoIndex])
        weights = np.array([int(matrix[y,2] == 1)*0+1 for y in treinoIndex],dtype="int32")        
        outputs = np.array([int(matrix[y,2] == 1) for y in treinoIndex],dtype="int32")
            
        model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
        model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)       
        
        testOutputs = [matrix[y,2] for y in testeIndex] 
        testInputs = np.array([matrix[y,:2]  for y in testeIndex])   
        predicts = model.predict(testInputs)
        predictsProbas = model.predict_proba(testInputs)
        scores[fold] = log_loss(testOutputs, predictsProbas)
        recall[fold] = recall_score(testOutputs, predicts)
        precision[fold] = specificity(testOutputs, predicts)        
        
    
    print("Final:"+ metric[0] + " = Error:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = Sensitive:%2.2e[+/- %2.2e]"%(np.mean(recall),np.std(recall)))
    print("Final:"+ metric[0] + " = specificity:%2.2e[+/- %2.2e]"%(np.mean(precision),np.std(precision)))       
    return np.mean(scores), np.std(scores)

In [21]:
metrics = [   ["neural network (2,2)",[2, 2]],["neural network (2,1 tanh,2)",[2, (1,"tanh"), 2]],["neural network (2,2 tanh,2)",[2, (2, 'tanh'), 2]]]


## Colesterol e Idade

In [22]:
inputOutput = np.array([np.concatenate((matrixProcessed[x,:2],[matrix[x,3] == 1]),axis=0) for x in range(0,len(matrix[:,3]))])
for metric in metrics:
    KFold(inputOutput, metric=metric)


Final:neural network (2,2) = Error:5.33e-01[+/- 2.99e-02]
Final:neural network (2,2) = Sensitive:4.78e-01[+/- 6.69e-02]
Final:neural network (2,2) = specificity:8.71e-01[+/- 4.90e-02]
Final:neural network (2,1 tanh,2) = Error:5.26e-01[+/- 6.20e-02]
Final:neural network (2,1 tanh,2) = Sensitive:5.79e-01[+/- 9.16e-02]
Final:neural network (2,1 tanh,2) = specificity:8.41e-01[+/- 3.46e-02]
Final:neural network (2,2 tanh,2) = Error:5.28e-01[+/- 5.26e-02]
Final:neural network (2,2 tanh,2) = Sensitive:5.82e-01[+/- 8.63e-02]
Final:neural network (2,2 tanh,2) = specificity:8.40e-01[+/- 4.86e-02]


## Colesterol e Glicemia

In [23]:
inputOutput = np.array([[matrixProcessed[x,0],matrixProcessed[x,2],matrix[x,3] == 1] for x in range(0,len(matrix[:,3]))])
for metric in metrics:
    KFold(inputOutput, metric=metric)
with open("cg.pkl",'wb') as f:
    np.savetxt(f, inputOutput, delimiter=',') 


Final:neural network (2,2) = Error:5.60e-01[+/- 4.94e-02]
Final:neural network (2,2) = Sensitive:4.31e-01[+/- 8.75e-02]
Final:neural network (2,2) = specificity:8.58e-01[+/- 3.68e-02]
Final:neural network (2,1 tanh,2) = Error:5.63e-01[+/- 3.78e-02]
Final:neural network (2,1 tanh,2) = Sensitive:4.33e-01[+/- 7.67e-02]
Final:neural network (2,1 tanh,2) = specificity:8.62e-01[+/- 3.13e-02]
Final:neural network (2,2 tanh,2) = Error:5.65e-01[+/- 6.44e-02]
Final:neural network (2,2 tanh,2) = Sensitive:4.50e-01[+/- 8.06e-02]
Final:neural network (2,2 tanh,2) = specificity:8.51e-01[+/- 5.01e-02]


## Glicemia e Idade

In [24]:
inputOutput = np.array([[matrixProcessed[x,1],matrixProcessed[x,2],matrix[x,3] == 1] for x in range(0,len(matrix[:,3]))])
for metric in metrics:
    KFold(inputOutput, metric=metric)

with open("ig.pkl",'wb') as f:
    np.savetxt(f, inputOutput, delimiter=',') 

Final:neural network (2,2) = Error:5.65e-01[+/- 3.12e-02]
Final:neural network (2,2) = Sensitive:4.28e-01[+/- 7.75e-02]
Final:neural network (2,2) = specificity:8.52e-01[+/- 4.36e-02]
Final:neural network (2,1 tanh,2) = Error:5.67e-01[+/- 5.11e-02]
Final:neural network (2,1 tanh,2) = Sensitive:4.58e-01[+/- 8.21e-02]
Final:neural network (2,1 tanh,2) = specificity:8.47e-01[+/- 4.55e-02]
Final:neural network (2,2 tanh,2) = Error:5.71e-01[+/- 4.66e-02]
Final:neural network (2,2 tanh,2) = Sensitive:4.37e-01[+/- 9.35e-02]
Final:neural network (2,2 tanh,2) = specificity:8.33e-01[+/- 6.91e-02]
